<a href="https://colab.research.google.com/github/9105507/UFV_VISUALIZACION_DE_DATOS/blob/main/Ejercicios_Clase/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica de Python (ventas en Europa)**

Preguntas que hay que responder (con gráficos):

1. Como se distribuyen las ventas realizadas en:
  * Cada país.
  * Cada mes y año.
  * Cada marca.

2. Cual es la tendencia y estacionalidad de:
  * Todas las ventas del país con menos venta.
  * La marca con más ventas.

3. Cuales son las predicciones hechas en España y como de
buenas son.

Se importan librerías usadas.

In [112]:
import pandas as pd
# import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose

Se cargan los datos.

In [113]:
data = pd.read_csv('datos_ejercicio_ventas.csv', delimiter=',')
data.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


## Análisis Previo

Tamaño de los datos.

In [114]:
data.shape

(18666, 8)

Info de los datos.

In [115]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18666 entries, 0 to 18665
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   COUNTRY        18666 non-null  object 
 1   SUBBRAND       18666 non-null  object 
 2   YEAR           18666 non-null  int64  
 3   MONTH          18666 non-null  int64  
 4   SCENARIO       18666 non-null  object 
 5   FORECAST       17766 non-null  object 
 6   FORECAST_YEAR  17766 non-null  float64
 7   AMOUNT         18666 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


DATASET:
CADA REGISTRO REPRESENTA LA CANTIDAD VENDIDA DE UN PRODUCTO EN UN PAÍS Y EN UN INSTANTE TEMPORAL (DADO EN MESES). ESTA CANTIDAD PUEDE SER UN DATO REAL O UNA PREDICCIÓN HECHA.
* 'COUNTRY': STRING, PAÍS, VARIABLE CATEGÓRICA.
* 'SUBBRAND': STRING, PRODUCTO, VARIABLE CATEGÓRICA.
* 'YEAR': INT, AÑO DEL VALOR, VARIABLE DISCRETA.
* 'MONTH': INT, MES DEL VALOR, VARIABLE DISCRETA.
* 'SCENARIO': STRING, INDICA SI EL VALOR ES REAL O UNA PREDICCIÓN, VARIABLE CATEGÓRICA.
* 'FORECAST': STRING, SI ES UNA PREDICCIÓN, INDICA EL PERÍODO DESDE EL QUE SE HACE LA PREDICCIÓN. MÁS CONCRETAMENTE, 'P0X' indica que se está en el final del mes X-1. LO INDICA A MES VENCIDO. 'P02' ES FINALES DE ENERO / PRINCIPIOS DE FEBRERO. VARIABLE CATEGORICA
* 'FORECAST_YEAR': FLOAT, EL AÑO DESDE EL QUE SE ESTÁ HACIENDO LA PREVISIÓN, VARIABLE DISCRETA.
* 'AMOUNT': INT, REPRESENTA UN NÚMERO: Nº PRODUCTOS VENDIDOS. VARIABLE CONTINUA.

Nº de Nan's por variable:

In [116]:
data.isna().sum()

COUNTRY            0
SUBBRAND           0
YEAR               0
MONTH              0
SCENARIO           0
FORECAST         900
FORECAST_YEAR    900
AMOUNT             0
dtype: int64

In [117]:
data[(data['FORECAST'].isna()) & (data['FORECAST_YEAR'].isna())]

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
277,Portugal,Pepsi Max (L3),2023,10,actual,NaN,NaN,1.885949e+05
278,Portugal,7up (L3),2023,3,actual,NaN,NaN,2.934971e+05
279,Portugal,7up (L3),2023,10,actual,NaN,NaN,3.484466e+05
280,Great Britain,7up Free (L3),2023,10,actual,NaN,NaN,1.172553e+06
281,Norway,Pepsi Regular (L3),2023,10,actual,NaN,NaN,3.784859e+04
...,...,...,...,...,...,...,...,...
18595,Great Britain,Lipton (L3),2023,9,actual,NaN,NaN,8.492634e+05
18596,Italy,Pepsi Regular (L3),2023,9,actual,NaN,NaN,1.562573e+06
18597,Great Britain,7up Free (L3),2023,9,actual,NaN,NaN,1.293228e+06
18598,Great Britain,Mountain Dew (L3),2023,9,actual,NaN,NaN,2.066463e+05


TODOS LOS NAN'S DE 'FORECAST' SON TAMBIÉN TODOS LOS NAN'S DE 'FORECAST_YEAR'

In [118]:
data[data['SCENARIO'] == 'actual']

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
277,Portugal,Pepsi Max (L3),2023,10,actual,NaN,NaN,1.885949e+05
278,Portugal,7up (L3),2023,3,actual,NaN,NaN,2.934971e+05
279,Portugal,7up (L3),2023,10,actual,NaN,NaN,3.484466e+05
280,Great Britain,7up Free (L3),2023,10,actual,NaN,NaN,1.172553e+06
281,Norway,Pepsi Regular (L3),2023,10,actual,NaN,NaN,3.784859e+04
...,...,...,...,...,...,...,...,...
18595,Great Britain,Lipton (L3),2023,9,actual,NaN,NaN,8.492634e+05
18596,Italy,Pepsi Regular (L3),2023,9,actual,NaN,NaN,1.562573e+06
18597,Great Britain,7up Free (L3),2023,9,actual,NaN,NaN,1.293228e+06
18598,Great Britain,Mountain Dew (L3),2023,9,actual,NaN,NaN,2.066463e+05


TODOS LOS NAN'S SE DAN CUANDO EL DATO ES REAL. ESTO TIENE SENTIDO, YA QUE SI TENGO EL DATO REAL, NO HAY PREDICCIÓN

Se juntan 'MONTH' y 'YEAR' en una sola variable 'DATE'.

In [119]:
# A las predicciones se les asigna el último día del mes.
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1)) + pd.offsets.MonthEnd(0)

# A los datos reales se les asigna el primer día del mes.
data.loc[data['SCENARIO'] == 'actual', 'DATE'] = pd.to_datetime(data.loc[data['SCENARIO'] == 'actual', ['YEAR', 'MONTH']].assign(DAY=1))

Se eliminan duplicados.

In [120]:
data = data.drop_duplicates()
data.shape

(18003, 9)

In [121]:
data[data['AMOUNT'] < 0]

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,DATE
6564,Hungary,7up (L3),2023,9,actual,NaN,NaN,-19481.652378,2023-09-01
7261,Hungary,7up (L3),2024,1,actual,NaN,NaN,-188.474017,2024-01-01
7904,Norway,Lipton (L3),2024,7,actual,NaN,NaN,-3177.460145,2024-07-01
8791,Hungary,7up (L3),2023,11,actual,NaN,NaN,-13549.371551,2023-11-01
11325,Hungary,7up (L3),2023,7,actual,NaN,NaN,-6332.708713,2023-07-01
11659,Norway,Lipton (L3),2023,9,actual,NaN,NaN,-217120.103133,2023-09-01
14006,Hungary,7up (L3),2023,12,actual,NaN,NaN,-10.791926,2023-12-01
15840,Norway,Lipton (L3),2024,3,actual,NaN,NaN,-173.296795,2024-03-01
17174,Hungary,7up (L3),2024,2,actual,NaN,NaN,-1.394871,2024-02-01
17719,Norway,Lipton (L3),2024,8,actual,NaN,NaN,-2435.665326,2024-08-01


Hay datos con 'AMOUNT' negativo. Se eliminan:

In [122]:
data = data[data['AMOUNT'] >= 0]

**COSAS INTERESANTES QUE ANALIZAR**:
1. Nº ACTUALS Y FORECAST.
2. HORIZONTE DE PREVISIÓN. PISTA: ES EL MISMO SIEMPRE.
3. Nº PAÍSES, PRODUCTOS.
4. HISTÓRICO DE DATOS DE ACTUALS / FORECAST.
5. Nº DE FORECAST DISTINTOS.

**1. Nº ACTUALS Y FORECAST.**

In [123]:
data[data['SCENARIO'] == 'actual'].shape[0]

890

HAY 900 DATOS REALES.

In [124]:
data[data['SCENARIO'] == 'AI_forecast'].shape[0]

17103

HAY 17103 VALORES PREDICHOS.

**2. HORIZONTE DE PREVISIÓN.**

El instante desde el que se predice para una marca de un país viene dado por la combinación de 'FORECAST' y 'FORECAST_YEAR'. Para ver cuál es el horizonte de previsión para cada predicción hecha, basta con ver cuántas combinaciones únicas de 'MONTH' y 'YEAR' (de momentos predichos) hay para cada combinación de ('FORECAST', 'FORECAST_YEAR', 'COUNTRY', 'SUBBRAND'). Así, se calculan los horizontes de previsión únicos en el dataset con el siguiente código.

In [125]:
df_grouped = data.groupby(['FORECAST', 'FORECAST_YEAR', 'COUNTRY', 'SUBBRAND'], observed=True)

unique_dates_count = df_grouped['DATE'].nunique().reset_index(name='UNIQUE_DATES_COUNT')
print(unique_dates_count.head())
print("\nLista de horizontes de previsión únicos:", unique_dates_count['UNIQUE_DATES_COUNT'].unique().tolist())

  FORECAST  FORECAST_YEAR COUNTRY            SUBBRAND  UNIQUE_DATES_COUNT
0  AI_P02F         2023.0   Czech            7up (L3)                  18
1  AI_P02F         2023.0   Czech         Lipton (L3)                  18
2  AI_P02F         2023.0   Czech   Mountain Dew (L3)                  18
3  AI_P02F         2023.0   Czech      Pepsi Max (L3)                  18
4  AI_P02F         2023.0   Czech  Pepsi Regular (L3)                  18

Lista de horizontes de previsión únicos: [18]


El horizonte de previsión es el mismo para todas las predicciones realizadas, siempre es 18.

**3. Nº PAÍSES, PRODUCTOS.**

In [126]:
len(data['COUNTRY'].unique().tolist())

9

In [127]:
data['COUNTRY'].unique().tolist()

['Portugal',
 'Great Britain',
 'Spain',
 'Hungary',
 'Norway',
 'Denmark',
 'Netherlands',
 'Italy',
 'Czech']

In [128]:
len(data['SUBBRAND'].unique().tolist())

6

In [129]:
data['SUBBRAND'].unique().tolist()

['Lipton (L3)',
 'Pepsi Max (L3)',
 '7up (L3)',
 'Pepsi Regular (L3)',
 'Mountain Dew (L3)',
 '7up Free (L3)']

**4. HISTÓRICO DE DATOS DE ACTUALS / FORECAST.**

Visualización de la serie temporal real de alguna de las combinaciones ('COUNTRY', 'SUBBRAND') junto con la predicción hecha desde un ('FORECAST', 'FORECAST_YEAR'). Si se quiere cambiar, hay que modificar los parámetros 'producto', 'pais', forecast y year en el código.

In [130]:
# Especificar parámetros
producto = '7up (L3)'
país = 'Czech'
forecast = 'AI_P04F'
year = 2023

# --- Filtrar y preparar datos de valores reales ---
# Filtrar el grupo de datos reales
df_real = data[(data['SCENARIO'] == 'actual') &
               (data['COUNTRY'] == país) &
               (data['SUBBRAND'] == producto)].copy()
df_real['SCENARIO'] = 'Valores Reales'  # Añadir etiqueta

# --- Filtrar y preparar datos de predicciones ---
# Filtrar el grupo de datos de predicción
df_forecast = data[(data['SCENARIO'] == 'AI_forecast') &
                   (data['COUNTRY'] == país) &
                   (data['SUBBRAND'] == producto) &
                   (data['FORECAST'] == forecast) &
                   (data['FORECAST_YEAR'] == year)].copy()
df_forecast['SCENARIO'] = 'Predicción'  # Añadir etiqueta

# Combinar ambos DataFrames
df_combined = pd.concat([df_real, df_forecast])

# Ordenar el DataFrame combinado por la columna 'DATE'
df_combined = df_combined.sort_values(by='DATE')

# Crear la gráfica combinada
fig = px.line(df_combined, x='DATE', y='AMOUNT', color='SCENARIO',
              title=f'Valores Reales y Predicciones para {producto} en {país} - Predicción desde {forecast} ({year})',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Cantidad'},
              markers=True, template='plotly')

# Mostrar la gráfica
fig.show()

**5. Nº DE FORECAST DISTINTOS.**

In [131]:
df_grouped = data.groupby(['FORECAST', 'FORECAST_YEAR', 'COUNTRY', 'SUBBRAND', 'DATE'], observed=True).size().unique().tolist()
print("Nº distintos de predicciones: ", df_grouped)

Nº distintos de predicciones:  [1, 2, 3, 4, 5, 10]


Esto significa que para un instante, en nuestra base de datos, hay casos en los que se predice 1 vez, otras se predice 2 veces para un mismo instante, 3, 4, 5 y hasta 10 veces para un mismo instante de tiempo.

## 1. Como se distribuyen las ventas realizadas en:
* Cada país
* Cada mes y año
* Cada marca

### Cada país.

In [132]:
# Filtra datos donde 'SCENARIO' es 'actual', agrupa por 'COUNTRY' y suma 'AMOUNT', luego ordena por 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['COUNTRY'], as_index=False)['AMOUNT'].sum().sort_values('AMOUNT')
# Visualizar el piechart
fig = px.pie(df_grouped, names='COUNTRY', values='AMOUNT',
             title='Pie Chart - Distribución de Ventas por País - Valores Totales por País',
             labels={'COUNTRY': 'País', 'AMOUNT': 'Cantidad Vendida'},
             color_discrete_sequence=px.colors.qualitative.G10)
# Mostrar la gráfica
fig.show()

In [133]:
# Filtra datos donde 'SCENARIO' es 'actual', agrupa por 'COUNTRY' y suma 'AMOUNT', luego ordena por 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['COUNTRY'], as_index=False)['AMOUNT'].sum().sort_values('AMOUNT')
# Visualizar el barplot
fig = px.bar(df_grouped, x='COUNTRY', y='AMOUNT',
             title='Bar Plot - Distribución de Ventas por País - Valores Totales por País',
             labels={'COUNTRY': 'País', 'AMOUNT': 'Cantidad Vendida'},
             color_discrete_sequence=px.colors.qualitative.G10)
# Mostrar la gráfica
fig.show()

In [134]:
# Orden deseado para los países
country_order = ['Spain', 'Portugal', 'Czech', 'Hungary', 'Italy', 'Norway', 'Denmark', 'Netherlands', 'Great Britain']
# Filtra los datos donde 'SCENARIO' es 'actual', agrupa por 'COUNTRY' y 'SUBBRAND', y suma 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['COUNTRY', 'SUBBRAND'], as_index=False)['AMOUNT'].sum()
# Convierte la columna 'COUNTRY' a tipo categórico con el orden especificado
df_grouped['COUNTRY'] = pd.Categorical(df_grouped['COUNTRY'], categories=country_order, ordered=True)
# Ordena el DataFrame por 'COUNTRY' y 'AMOUNT'
df_grouped = df_grouped.sort_values(['COUNTRY', 'AMOUNT'])
# Orden para barras ordenadas
df_grouped['ORDER'] = df_grouped.groupby('COUNTRY', observed=True).cumcount() + 1  # Esto genera un número de orden secuencial para cada país
# Visualizar el barplot
fig = px.bar(df_grouped, x='ORDER', y='AMOUNT', color='SUBBRAND',
             title='Bar Plot - Distribución de Ventas por País - Comparación entre Marcas',
             labels={'COUNTRY': 'País', 'AMOUNT': 'Cantidad Vendida'},
             color_discrete_sequence=px.colors.qualitative.G10,
             log_y=True, barmode='group', facet_col='COUNTRY')
# Se renombra el eje x
fig.update_xaxes(title_text='Spain', row=1, col=1)
fig.update_xaxes(title_text='Portugal', row=1, col=2)
fig.update_xaxes(title_text='Czech', row=1, col=3)
fig.update_xaxes(title_text='Hungary', row=1, col=4)
fig.update_xaxes(title_text='Italy', row=1, col=5)
fig.update_xaxes(title_text='Norway', row=1, col=6)
fig.update_xaxes(title_text='Denmark', row=1, col=7)
fig.update_xaxes(title_text='Netherlands', row=1, col=8)
fig.update_xaxes(title_text='Great Britain', row=1, col=9)
fig.for_each_xaxis(lambda xaxis: xaxis.update(tickvals=[], ticktext=[]))
# Mostrar la gráfica
fig.show()

### Cada mes y año.

In [135]:
# Convierte la columna 'YEAR' a tipo de dato categórico
data['YEAR'] = data['YEAR'].astype('category')
# Filtra los datos donde 'SCENARIO' es 'actual', agrupa por 'YEAR' y suma 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['YEAR'], as_index=False, observed=True)['AMOUNT'].sum()
# Visualizar el barplot
fig = px.bar(df_grouped, x='YEAR', y='AMOUNT',
    title='Bar Plot - Distribución de Ventas por Año - Valores Totales por Año',
    labels={'YEAR': 'Año', 'AMOUNT': 'Cantidad Vendida'},
    color_discrete_sequence=px.colors.qualitative.G10)
# Ajuste en la distancia en el eje x
fig.update_xaxes(dtick=1)
# Mostrar el gráfico
fig.show()

In [136]:
# Convierte la columna 'YEAR' a tipo de dato categórico
data['YEAR'] = data['YEAR'].astype('category')
# Filtra los datos donde 'SCENARIO' es 'actual', agrupa por 'YEAR' y suma 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['YEAR'], as_index=False, observed=True)['AMOUNT'].sum()
# Crear un gráfico de pastel
fig = px.pie(df_grouped, names='YEAR', values='AMOUNT',
             title='Pie Chart - Distribución de Ventas por País',
             labels={'YEAR': 'Año', 'AMOUNT': 'Cantidad Vendida'},
             color_discrete_sequence=px.colors.qualitative.G10)
# Mostrar el gráfico
fig.show()

In [137]:
# Convierte la columna 'MONTH' a tipo de dato categórico
data['MONTH'] = data['MONTH'].astype('category')
# Filtra los datos donde 'SCENARIO' es 'actual', agrupa por 'MONTH' y suma 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['MONTH'], as_index=False, observed=True)['AMOUNT'].sum()
# Visualizar el barplot
fig = px.bar(df_grouped, x='MONTH', y='AMOUNT',
    title='Bar Plot - Distribución de Ventas por Mes - Valores Totales por Mes',
    labels={'MONTH': 'Mes', 'AMOUNT': 'Cantidad Vendida'},
    color_discrete_sequence=px.colors.qualitative.G10)
# Ajuste en la distancia en el eje x
fig.update_xaxes(dtick=1)
# Mostrar el gráfico
fig.show()

In [138]:
# Convierte la columna 'YEAR' a tipo de dato categórico
data['YEAR'] = data['YEAR'].astype('category')
# Convierte la columna 'MONTH' a tipo de dato categórico
data['MONTH'] = data['MONTH'].astype('category')
# Filtra los datos donde 'SCENARIO' es 'actual', agrupa por 'YEAR' y 'MONTH', y suma 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['YEAR', 'MONTH'], as_index=False, observed=True)['AMOUNT'].sum()
# Visualizar el barplot
fig = px.bar(df_grouped, x='MONTH', y='AMOUNT', color='YEAR',
    title='Bar Plot - Distribución de Ventas por Mes y Año - Comparación 2023 vs 2024',
    labels={'MONTH': 'Mes', 'AMOUNT': 'Cantidad Vendida', 'YEAR': 'Año'},
    color_discrete_sequence=px.colors.qualitative.G10, barmode='group')
# Ajuste en la distancia en el eje x
fig.update_xaxes(dtick=1)
# Mostrar el gráfico
fig.show()

### Cada marca.

In [139]:
# Filtra los datos donde 'SCENARIO' es 'actual', agrupa por 'SUBBRAND', y suma y ordena por 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['SUBBRAND'], as_index=False, observed=True)['AMOUNT'].sum().sort_values('AMOUNT')
# Visualizar el piechart
fig = px.pie(df_grouped, names='SUBBRAND', values='AMOUNT',
             title='Pie Chart - Distribución de Ventas por Marca',
             labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Cantidad Vendida'},
             color_discrete_sequence=px.colors.qualitative.G10)
# Mostrar el gráfico
fig.show()

In [140]:
# Filtra los datos donde 'SCENARIO' es 'actual', agrupa por 'SUBBRAND', y suma y ordena por 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['SUBBRAND'], as_index=False, observed=True)['AMOUNT'].sum().sort_values('AMOUNT')
# Visualizar el barplot
fig = px.bar(df_grouped, x='SUBBRAND', y='AMOUNT',
    title='Bar Plot - Distribución de Ventas por Marca - Valores Totales por Marca',
    labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Cantidad Vendida'},
    color_discrete_sequence=px.colors.qualitative.G10)
# Ajuste en la distancia en el eje x
fig.update_xaxes(dtick=1)
# Mostrar el gráfico
fig.show()

In [141]:
# Filtra los datos donde 'SCENARIO' es 'actual', agrupa por 'SUBBRAND' y 'YEAR', y suma 'AMOUNT'
df_grouped = data[data['SCENARIO'] == 'actual'].groupby(['SUBBRAND', 'YEAR'], as_index=False, observed=True)['AMOUNT'].sum()
# Visualizar el barplot
fig = px.bar(df_grouped, x='SUBBRAND', y='AMOUNT', color = 'YEAR',
    title='Distribución de Ventas por Marca - Comparación 2023 vs 2024',
    labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Cantidad Vendida'},
    color_discrete_sequence=px.colors.qualitative.G10, barmode='group',
    category_orders={'SUBBRAND': ['Mountain Dew (L3)', '7up (L3)', '7up Free (L3)', 'Lipton (L3)', 'Pepsi Regular (L3)', 'Pepsi Max (L3)']})
# Ajuste en la distancia en el eje x
fig.update_xaxes(dtick=1)
# Mostrar el gráfico
fig.show()

## 2. Cual es la tendencia y estacionalidad de:
  * Todas las ventas del país con menos venta.
  * La marca con más ventas.

### País con menos venta.

In [142]:
# Se encuentra el país con menos ventas.
df_grouped = data[data['SCENARIO'] == 'actual'].groupby('COUNTRY', as_index=False)['AMOUNT'].sum()
pais = df_grouped.sort_values('AMOUNT', ascending=True).head(1)['COUNTRY'].values[0]
# Se encuentra la serie temporal de valores reales para el país con menos ventas
df_grouped = data[(data['SCENARIO'] == 'actual') & (data['COUNTRY'] == pais)].groupby('DATE')['AMOUNT'].sum()
df_grouped.index = pd.to_datetime(df_grouped.index)
df_grouped = df_grouped.asfreq('MS')
# Descomposición de la serie en tendencia, estacionalidad y residuo
decompose_result = seasonal_decompose(df_grouped, model='additive', period=4)
# Crear la figura
fig = go.Figure()
# Agregar la serie temporal original
fig.add_scatter(x=df_grouped.index, y=df_grouped, mode='lines+markers',
                name='Serie Temporal Original', line=dict(color='blue'))
# Agregar la tendencia
fig.add_scatter(x=df_grouped.index, y=decompose_result.trend, mode='lines+markers',
                name='Tendencia', line=dict(color='green'))
# Agregar la estacionalidad
fig.add_scatter(x=df_grouped.index, y=decompose_result.seasonal, mode='lines+markers',
                name='Estacionalidad', line=dict(color='orange'))
# Configuración de la leyenda y el título
fig.update_layout(title=f'Serie Temporal de Ventas de {pais} junto con su Tendencia y su Estacionalidad',
                 xaxis_title='Date', yaxis_title='Amount', template='plotly')
# Mostrar la gráfica
fig.show()

Tendencia creciente leve, muy poca estacionalidad.

### Marca con más ventas.

In [143]:
# Se encuentra el país con menos ventas.
df_grouped = data[data['SCENARIO'] == 'actual'].groupby('SUBBRAND', as_index=False)['AMOUNT'].sum()
marca = df_grouped.sort_values('AMOUNT', ascending=False).head(1)['SUBBRAND'].values[0]
# Se encuentra la serie temporal de valores reales para la marca con más ventas
df_grouped = data[(data['SCENARIO'] == 'actual') & (data['SUBBRAND'] == marca)].groupby('DATE')['AMOUNT'].sum()
df_grouped.index = pd.to_datetime(df_grouped.index)
df_grouped = df_grouped.asfreq('MS')
# Descomposición de la serie en tendencia, estacionalidad y residuo
decompose_result = seasonal_decompose(df_grouped, model='additive', period=4)
# Crear la figura
fig = go.Figure()
# Agregar la serie temporal original
fig.add_scatter(x=df_grouped.index, y=df_grouped, mode='lines+markers',
                name='Serie Temporal Original', line=dict(color='blue'))
# Agregar la tendencia
fig.add_scatter(x=df_grouped.index, y=decompose_result.trend, mode='lines+markers',
                name='Tendencia', line=dict(color='green'))
# Agregar la estacionalidad
fig.add_scatter(x=df_grouped.index, y=decompose_result.seasonal, mode='lines+markers',
                name='Estacionalidad', line=dict(color='orange'))
# Configuración de la leyenda y el título
fig.update_layout(title=f'Serie Temporal de Ventas de {marca} junto con su Tendencia y su Estacionalidad',
                 xaxis_title='Date', yaxis_title='Amount', template='plotly')
# Mostrar la gráfica
fig.show()

Tendencia creciente leve, estacionalidad casi nula.

## 3. Cuales son las predicciones hechas en España y como de buenas son.

Hay varias series temporales que representan las predicciones hechas para España, las hechas desde: finales de diciembre de 2022 (AI_PF, 2023), finales de enero de 2023 (AI_P02F, 2023), finales de febrero de 2023 (AI_P03F, 2023), etc. Por esto, se presentan varios gráficos, uno con todas las predicciones junto con los datos reales de España, y otra en la que se puede decidir cuál se quiere comparar con los datos reales, y en la que también se podrá visualizar el error cometido.

In [144]:
# Se encuentra la serie temporal de valores reales para España
df_grouped = data[(data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain')].groupby('DATE')['AMOUNT'].sum()
df_grouped.index = pd.to_datetime(df_grouped.index)
df_grouped = df_grouped.asfreq('MS')

# Crear la figura
fig = go.Figure()

# Agregar la serie temporal original
fig.add_scatter(x=df_grouped.index, y=df_grouped, mode='lines+text',
                name='Serie Temporal Original', line=dict(color='black', width=4))

# Crear el DataFrame agrupado y definir el orden de 'FORECAST'
forecast_order = ['AI_PF', 'AI_P02F', 'AI_P03F', 'AI_P04F', 'AI_P05F',
                  'AI_P06F', 'AI_P07F', 'AI_P08F', 'AI_P09F', 'AI_P10F',
                  'AI_P11F', 'AI_P12F']
data['FORECAST'] = pd.Categorical(data['FORECAST'], categories=forecast_order, ordered=True)
df_grouped = data[(data['SCENARIO'] == 'AI_forecast') & (data['COUNTRY'] == 'Spain')].groupby(['FORECAST', 'FORECAST_YEAR', 'DATE'], observed = True)['AMOUNT'].sum()

# Lista de colores para cambiar con cada iteración
colors = px.colors.qualitative.G10 + px.colors.qualitative.D3_r

# Iterar sobre los grupos únicos de 'FORECAST' y 'FORECAST_YEAR'
for i, ((forecast, forecast_year), group) in enumerate(df_grouped.groupby(level=['FORECAST', 'FORECAST_YEAR'], observed=True)):
    # Serie temporales de predicciones para la combinación de la iteración i
    group = group.reset_index().sort_values(by='DATE')  # Ordenar por 'DATE' y reiniciar índice
    group.set_index('DATE', inplace=True)
    group.index = pd.to_datetime(group.index)  # Convertir índice a datetime
    group = group.asfreq('ME')  # Establecer la frecuencia mensual

    # Agregar la serie temporal de predicciones a la figura
    fig.add_scatter(
        x=group.index, y=group['AMOUNT'], mode='lines+text',
        name=f'Forecast - {forecast} ({int(forecast_year)})',
        line=dict(color=colors[i % len(colors)], width=1.5))

# Configuración de la leyenda y el título
fig.update_layout(title=f'Serie Temporal de Ventas en España junto con TODAS las predicciones hechas.',
                 xaxis_title='Date', yaxis_title='Amount', template='plotly')

# Mostrar la gráfica
fig.show()

Este gráfico es caótico, y quiero remarcar que es un gráfico exploratorio, el cuál lo dejo debido a que considero que tiene un gran valor ya que me ha ayudado a comprender los datos y el cuál me ha servido para construir los siguientes, que esos ya no son exploratorios.

Comparación de la serie temporal de datos reales de España con las predicciones hechas desde un 'forecast' de un 'forecast_year' para cada combinación. Se grafican junto a un histograma que representa la distribución de los residuos en valor absoluto para cada combinación.

In [145]:
forecast_year = 2023.0  # Especifica el año de forecast deseado
forecast_order = ['AI_PF', 'AI_P02F', 'AI_P03F', 'AI_P04F', 'AI_P05F', 'AI_P06F', 'AI_P07F', 'AI_P08F', 'AI_P09F', 'AI_P10F', 'AI_P11F', 'AI_P12F']
for forecast in forecast_order:
    # Se obtiene la serie temporal de datos reales
    df_grouped = data[(data['SCENARIO'] == 'actual') & (data['COUNTRY'] == 'Spain')].groupby('DATE')['AMOUNT'].sum()
    df_grouped.index = pd.to_datetime(df_grouped.index)
    df_grouped = df_grouped.asfreq('MS')

    # Crear la figura
    fig = go.Figure()

    # Agregar la serie temporal original
    fig.add_scatter(x=df_grouped.index, y=df_grouped, mode='lines+markers',
                    name='Serie Temporal Original', line=dict(color='blue'))

    # Filtrar el DataFrame y agrupar por 'FORECAST', 'FORECAST_YEAR', y 'DATE' para la predicción
    df_forecast = data[(data['SCENARIO'] == 'AI_forecast') & (data['COUNTRY'] == 'Spain')].groupby(['FORECAST', 'FORECAST_YEAR', 'DATE'], observed=True)['AMOUNT'].sum()

    # Obtener la serie temporal de predicciones
    group = df_forecast.loc[(forecast, forecast_year)].reset_index().sort_values(by='DATE')  # Ordenar por 'DATE' y reiniciar índice
    group.set_index('DATE', inplace=True)
    group.index = pd.to_datetime(group.index)  # Convertir índice a datetime
    group = group.asfreq('ME')  # Establecer la frecuencia mensual

    # Agregar la serie temporal de predicción a la figura
    fig.add_scatter(
        x=group.index, y=group['AMOUNT'], mode='lines+markers',
        name=f'Forecast - {forecast} ({int(forecast_year)})',
        line=dict(color='green'))

    # Código que grafica la diferencia entre valores reales y valores predichos.
    desfase = group.index[0].month - df_grouped.index[0].month + 1
    i = 0
    residuos = []
    for date in group.index:
        if (i+desfase) > len(df_grouped) - 1:
            break
        if (df_grouped.index[i+desfase] - date) == pd.Timedelta(days=1):
            # Error Porcentual
            residuos.append(abs((df_grouped.iloc[i+desfase] - group.loc[date, 'AMOUNT']) / df_grouped.iloc[i+desfase]))
            # Agregar una línea que conecte el valor original con el de la predicción
            fig.add_scatter(
                x=[date, date],
                y=[df_grouped.iloc[i+desfase], group.loc[date, 'AMOUNT']],
                mode='lines',
                line=dict(color='red', dash='dot'),
                showlegend=False  # No agregar estas líneas a la leyenda
            )
        i += 1

    # Configuración de la leyenda y el título
    fig.update_layout(title=f'Serie Temporal de Ventas en España vs Predicciones {forecast} ({int(forecast_year)})',
                    xaxis_title='Date', yaxis_title='Amount', template='plotly')

    # Mostrar la gráfica
    fig.show()

    # DataFrame de residuos
    df_errores = {'RESIDUOS': residuos}

    # Crear el histograma
    fig = px.histogram(df_errores, x='RESIDUOS', nbins=5, title=f'Histograma - Distribución de los Residuos en Valor Absoluto para Predicciones {forecast} ({int(forecast_year)})',
                    labels={'RESIDUOS': 'Residuos Porcentual (%)'}, template='plotly', opacity = 0.6,
                    color_discrete_sequence=['#FF0000'])

    # Actualizar trazas para agregar bordes a las barras
    fig.update_traces(marker_line_color='black', marker_line_width=2)  # Ajustar color y ancho del borde
    fig.show()

En general, las predicciones son malas, con altos valores absolutos para los residuos.